In [ ]:
import gradio as gr
import pandas as pd
import random
import tempfile
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
# Load the IBM Granite LLM model
model_name = "ibm-granite/granite-3.3-2b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
# Global variable to store feedback data
feedback_data = pd.DataFrame(columns=["Issue", "Category", "Timestamp"])

def generate_response(prompt, max_length=1000):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=max_length)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def waste_management(item):
    prompt = f"Provide detailed disposal instructions for {item}, including recycling options if applicable."
    return generate_response(prompt)

def energy_saving(habits):
    prompt = f"Based on these energy habits: {habits}. Provide comprehensive energy-efficient practices with explanations."
    return generate_response(prompt)

def collect_feedback(issue, category):
    global feedback_data
    new_entry = pd.DataFrame({
        "Issue": [issue],
        "Category": [category],
        "Timestamp": [pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S")]
    })
    feedback_data = pd.concat([feedback_data, new_entry], ignore_index=True)
    base_response = f"✅ Feedback received!\n\nIssue: {issue}\nCategory: {category}\n\nTotal feedback entries: {len(feedback_data)}"
    with tempfile.NamedTemporaryFile(delete=False, suffix=".csv") as tmp:
        feedback_data.to_csv(tmp.name, index=False)
        download_path = tmp.name
    return base_response, download_path

def green_challenge():
    challenges = [
        "Avoid plastic bags today. Bring reusable bags when shopping.",
        "Take a 30-minute walk instead of driving for short distances.",
        "Turn off all non-essential lights and electronics for one hour this evening.",
        "Avoid using disposable coffee cups. Bring your own reusable cup.",
        "Spend 15 minutes collecting litter in your neighborhood.",
        "Use only reusable water bottles today - no single-use plastic bottles.",
        "Visit a local farmer's market and purchase one locally grown item."
    ]
    return random.choice(challenges)

def summarize_policy(document):
    prompt = f"Please provide a comprehensive summary of this policy document: {document}"
    return generate_response(prompt)

def eco_tips(keyword):
    prompt = f"Provide 5 detailed sustainability tips related to {keyword}."
    return generate_response(prompt)

def chat_assistant(question):
    prompt = f"User  asked: {question}\n\nProvide a detailed, comprehensive response with actionable sustainability strategies."
    return generate_response(prompt, max_length=1500)

In [ ]:
# Create Gradio Interface with improved examples
with gr.Blocks(theme=gr.themes.Soft()) as app:
    gr.Markdown("""
    # 🌱 Sustainable Smart City Assistant
    AI-powered urban sustainability toolkit with complete output visibility
    """)

    with gr.Tabs():
        # Tab 1: Waste Management
        with gr.Tab("♻ Waste Management"):
            with gr.Row():
                with gr.Column():
                    item_input = gr.Textbox(
                        label="Item to dispose of",
                        placeholder="e.g., old mobile phone, broken tube light"
                    )
                    waste_btn = gr.Button("Get Disposal Guide")
                with gr.Column():
                    waste_output = gr.Textbox(label="Disposal Instructions", lines=20, interactive=False)
            waste_btn.click(waste_management, inputs=item_input, outputs=waste_output)

        # Tab 2: Energy Savings
        with gr.Tab("⚡ Energy Savings"):
            with gr.Row():
                with gr.Column():
                    habits_input = gr.Textbox(
                        label="Describe your energy habits",
                        lines=3,
                        placeholder="e.g., leave fans and lights on all day"
                    )
                    energy_btn = gr.Button("Get Recommendations")
                with gr.Column():
                    energy_output = gr.Textbox(label="Energy Saving Tips", lines=20, interactive=False)
            energy_btn.click(energy_saving, inputs=habits_input, outputs=energy_output)

        # Tab 3: Feedback Collection
        with gr.Tab("📝 Feedback Collection"):
            with gr.Row():
                with gr.Column():
                    issue_input = gr.Textbox(
                        label="Issue description",
                        placeholder="e.g., streetlight not working in Sector 5"
                    )
                    category_input = gr.Textbox(
                        label="Category (e.g., waste, water)",
                        placeholder="e.g., electricity, water, sanitation"
                    )
                    feedback_btn = gr.Button("Submit Feedback")
                with gr.Column():
                    feedback_output = gr.Textbox(label="Feedback Status", lines=10, interactive=False)
                    download_button = gr.DownloadButton("Download All Feedback", visible=False)

            def handle_feedback(issue, category):
                response, download_path = collect_feedback(issue, category)
                return response, gr.DownloadButton("Download All Feedback", visible=True, value=download_path)

            feedback_btn.click(
                handle_feedback,
                inputs=[issue_input, category_input],
                outputs=[feedback_output, download_button]
            )

        # Tab 4: Green Challenge
        with gr.Tab("🌿 Daily Challenge"):
            with gr.Row():
                challenge_btn = gr.Button("Get Today's Eco Challenge")
                challenge_output = gr.Textbox(label="Your Challenge", interactive=False)
            challenge_btn.click(green_challenge, outputs=challenge_output)

        # Tab 5: Policy Summarizer
        with gr.Tab("📑 Policy Summary"):
            with gr.Row():
                with gr.Column():
                    policy_input = gr.Textbox(
                        label="Policy text",
                        lines=5,
                        placeholder="Paste your policy document here..."
                    )
                    policy_btn = gr.Button("Summarize")
                with gr.Column():
                    policy_output = gr.Textbox(label="Summary", lines=20, interactive=False)
            policy_btn.click(summarize_policy, inputs=policy_input, outputs=policy_output)

        # Tab 6: Eco Tips
        with gr.Tab("💡 Eco Tips"):
            with gr.Row():
                with gr.Column():
                    keyword_input = gr.Textbox(
                        label="Topic (e.g., recycling, energy)",
                        placeholder="e.g., solar energy, composting"
                    )
                    tips_btn = gr.Button("Get Tips")
                with gr.Column():
                    tips_output = gr.Textbox(label="Sustainability Tips", lines=20, interactive=False)
            tips_btn.click(eco_tips, inputs=keyword_input, outputs=tips_output)


In [ ]:
# Launch the app
app.launch()